# Benchmarking using custom structures

This notebook will demonstrate how to use custom structures in the MaterialsCoord benchmark.

*Written using:*
- MaterialsCoord==0.1.0

*Authors: Alex Ganose (10/14/19)*

---

## The `"coordination"` site property

MaterialsCoord uses pymatgen `Structure` objects to internally store the crystal structure data. The structures provided in the default structure groups contain an additional site property called `"coordination"`. This property contains information on the human interpreted coordination environment for each site.

For example, if a site is bonded to two oxygen and three chlorine atoms, the coordination property should be a dictionary of ``{"O": 2, "Cl": 3}``. If multiple coordination numbers are deemed correct these should be provided as a list. For example, if the coordination to gallium could be 4 or 12 coordinate, the coordination property should a dictionary of ``{"Ga": [4, 12]}``.
            
To see this in action, let's first load the benchmark with the elemental structure group.

In [1]:
from materialscoord.core import Benchmark

bm = Benchmark.from_structure_group("elemental")

If we examine the `"Ga_12174"` structure we see it only contains two atoms.

In [2]:
bm.structures["Ga_12174"]

Structure Summary
Lattice
    abc : 2.813 2.813 4.452
 angles : 90.0 90.0 90.0
 volume : 35.22853798800001
      A : 2.813 0.0 1.7224657230007525e-16
      B : -1.7224657230007525e-16 2.813 1.7224657230007525e-16
      C : 0.0 0.0 4.452
PeriodicSite: Ga (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Ga (1.4065, 1.4065, 2.2260) [0.5000, 0.5000, 0.5000]

The coordination of these sites can be accessed through the `site_properties` attribute.

In [3]:
bm.structures["Ga_12174"].site_properties

{'coordination': [{'Ga': [4.0, 12.0]}, {'Ga': [4.0, 12.0]}]}

Here we can see that each site has two acceptable coordination environments, namely the sites can be coordinated to either 4 or 12 Ga atoms.

## Preparing custom structures

Using custom structures with MaterialsCoord is easy. The only requirement is that you add the `"coordination"` site property to the structures with the desired "correct" bonding interpretation. Let's create a custom structure and prepare it for use with MaterialsCoord.

We'll recreate the NaCl face-centered cubic structure from scratch. Alternatively, structures can be loaded from files (CIF, POSCAR, etc...) using the `Structure.from_file()` method.

In [4]:
from pymatgen import Structure

structure = Structure.from_spacegroup(
    225,
    [[5.7, 0, 0], [0, 5.7, 0], [0, 0, 5.7]], 
    ["Na1+", "Cl1-"], 
    [[0, 0, 0], [0.5, 0, 0]]
)

structure

Structure Summary
Lattice
    abc : 5.7 5.7 5.7
 angles : 90.0 90.0 90.0
 volume : 185.193
      A : 5.7 0.0 0.0
      B : 0.0 5.7 0.0
      C : 0.0 0.0 5.7
PeriodicSite: Na+ (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Na+ (0.0000, 2.8500, 2.8500) [0.0000, 0.5000, 0.5000]
PeriodicSite: Na+ (2.8500, 0.0000, 2.8500) [0.5000, 0.0000, 0.5000]
PeriodicSite: Na+ (2.8500, 2.8500, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: Cl- (2.8500, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
PeriodicSite: Cl- (0.0000, 2.8500, 0.0000) [0.0000, 0.5000, 0.0000]
PeriodicSite: Cl- (0.0000, 0.0000, 2.8500) [0.0000, 0.0000, 0.5000]
PeriodicSite: Cl- (2.8500, 2.8500, 2.8500) [0.5000, 0.5000, 0.5000]

In the NaCl structure, each Na is bonded to 6 Cl, whereas each Cl is bonded to 6 Na.

We can therefore define our coordination dictionaries as:

In [5]:
na_coordination = {"Cl": 6}
cl_coordination = {"Na": 6}

In the structure we have generated, the first 4 atoms are Na and the last 4 atoms are Cl. We can therefore define the coordination for each site in the structure as:

In [6]:
all_sites_coordination = [na_coordination] * 4 + [cl_coordination] * 4

Note that that the length of `all_sites_coordination` must be the same as the number of atoms in the structure.

Finally, we can add the property to the structure using `add_site_property()` function.

In [7]:
structure.add_site_property("coordination", all_sites_coordination)

We can check that the `"coordination"` property was added correctly by examining the `site_properties` attribute:

In [8]:
structure.site_properties

{'coordination': [{'Cl': 6},
  {'Cl': 6},
  {'Cl': 6},
  {'Cl': 6},
  {'Na': 6},
  {'Na': 6},
  {'Na': 6},
  {'Na': 6}]}

## Running the benchmark on custom structures

To run the benchmark on custom structures, we can initialize the `Benchmark` object directly, **without** using the `from_structure_groups()` method.

The `Benchmark` object takes a dictionary as input, where the dictionary is formatted as `{"structure_name": Structure}`. Remember, each structure must have the `"coordination"` site property to be used a benchmark structure.

Let's initialize the benchmark using our custom structure:

In [9]:
structures_dict = {"NaCl_custom": structure}

bm = Benchmark(structures_dict)

We can then use the benchmark object like normal. Let's try benchmarking some near neighbor algorithms:

In [10]:
from pymatgen.analysis.local_env import BrunnerNN_reciprocal, VoronoiNN, EconNN

nn_methods = [BrunnerNN_reciprocal(), VoronoiNN(tol=0.5), EconNN()]

bm.score(nn_methods)

,BrunnerNN_reciprocal,VoronoiNN,EconNN
NaCl_custom,0.0,0.0,12.0
Total,0.0,0.0,12.0
